<a href="https://colab.research.google.com/github/BrennanB572/Home_Sales/blob/main/Home_Sales_starter_code_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install pyspark

In [17]:
spark_version = 'spark-3.4.0'
!wget -q https://archive.apache.org/dist/spark/$spark_version/$spark_version-bin-hadoop3.tgz

In [19]:
!tar xf $spark_version-bin-hadoop3.tgz

In [21]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

In [22]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
# spark_version = 'spark-3.4.0'
# os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
#!apt-get update
#!apt-get install openjdk-11-jdk-headless -qq > /dev/null
##!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
#!tar xf $SPARK_VERSION-bin-hadoop3.tgz
#!pip install -q findspark

# Set Environment Variables
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
#os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

In [23]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [27]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
path = SparkFiles.get("home_sales_revised.csv")
temp_df = spark.read.csv(path, header=True, inferSchema=True)
temp_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [28]:
# 2. Create a temporary view of the DataFrame.
temp_df.createOrReplaceTempView("home_sales")

In [31]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = spark.sql("SELECT round(avg(price), 2) as avg_price_per_4_bd_home FROM home_sales WHERE bedrooms = 4")
query.show()


+-----------------------+
|avg_price_per_4_bd_home|
+-----------------------+
|              299661.01|
+-----------------------+



In [32]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query = spark.sql("SELECT round(avg(price), 2) as avg_price_per_3_bd_3_bth_home FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3")
query.show()


+-----------------------------+
|avg_price_per_3_bd_3_bth_home|
+-----------------------------+
|                    292003.61|
+-----------------------------+



In [38]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query = spark.sql("SELECT year, avg_price FROM (SELECT extract(year from date) as year, round(avg(price), 2) as avg_price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 group by 1) order by 1 desc")
query.show()


+----+---------+
|year|avg_price|
+----+---------+
|2022|290242.99|
|2021|296330.96|
|2020|292289.09|
|2019|289859.14|
+----+---------+



In [39]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = spark.sql("SELECT view, avg_price FROM (SELECT view, round(avg(price), 2) as avg_price FROM home_sales group by 1) WHERE avg_price >= 350000 order by 1 desc")
query.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.0354571342468262 seconds ---


In [40]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [41]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [42]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

query = spark.sql("SELECT view, avg_price FROM (SELECT view, round(avg(price), 2) as avg_price FROM home_sales group by 1) WHERE avg_price >= 350000 order by 1 desc")
query.show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 2.069277048110962 seconds ---


In [43]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
temp_df.write.partitionBy("date_built").parquet("dates_built")

In [45]:
# 11. Read the parquet formatted data.
parqueted_df = spark.read.parquet("dates_built")
parqueted_df

DataFrame[id: string, date: date, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int, date_built: int]

In [46]:
# 12. Create a temporary table for the parquet data.
parqueted_df.createOrReplaceTempView("dates_built")

In [47]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query = spark.sql("SELECT view, avg_price FROM (SELECT view, round(avg(price), 2) as avg_price FROM home_sales group by 1) WHERE avg_price >= 350000 order by 1 desc")
query.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.6572654247283936 seconds ---


In [48]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [49]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False